## Spark Streaming
- 스트리밍으로 데이터를 받아서 rdd, data frame 으로  차곡차곡 정제해서 가져다 놓는 형태 
- Creating Driver : StreamingContext 

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.streaming import StreamingContext

In [3]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [4]:
# sparkContext, batchDuration : 몇 초마다 받겠다
# A StreamingContext represents the connection to a Spark cluster, and can be used to create DStream various input sources.
streaming = StreamingContext(sc,5)

In [5]:
streaming

In [6]:
# 원격에서 받아옴 but 지금은 localhost 에서 받아옴 
# 텍스트를 5초마다 받아옴 
lines = streaming.socketTextStream("localhost", 9999)

In [7]:
# Dstream --> rdd 형태 
type(lines)

pyspark.streaming.dstream.DStream

In [8]:
words = lines.flatMap(lambda line:line.split())

In [9]:
# 어떤단어(w) 가 몇번 나왔다는 식으로 --> (단어, 1)
pairs = words.map(lambda w:(w,1))

In [10]:
# 단어에 따라서 value 값들을 더함 
count = pairs.reduceByKey(lambda x,y:x+y)

In [11]:
count.pprint()

In [12]:
# nc 켜주고 실행

In [ ]:
streaming.start()
streaming.awaitTermination()

-------------------------------------------
Time: 2018-08-06 14:14:35
-------------------------------------------

-------------------------------------------
Time: 2018-08-06 14:14:40
-------------------------------------------

-------------------------------------------
Time: 2018-08-06 14:14:45
-------------------------------------------

-------------------------------------------
Time: 2018-08-06 14:14:50
-------------------------------------------

-------------------------------------------
Time: 2018-08-06 14:14:55
-------------------------------------------



### Structured Streaming
- use DataFrame
- RDD 에서는 row 
- DataFrame 을이용해 구조적 스트리밍 --> data structure --> 기존에 입력한 값도 보여줌 

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [5]:
# format 를 scoket 
lines = spark.readStream.format("socket")\
    .option("host","localhost")\
    .option("port", 9999)\
    .load()

In [6]:
from pyspark.sql.functions import explode, split

In [7]:
type(lines)

pyspark.sql.dataframe.DataFrame

In [10]:
# explode : Returns a new row for each element in the given array or map.
# alias : column 명을 붙여줌 'word' 라고
# word 라는 컬럼명 안에 row 단위로 쌓아 놓음 , 즉 하나의 테이블이랑 똑같음 
words = lines.select(
    explode(split(lines.value, " ")).alias("word")
)

In [11]:
wordCount = words.groupBy("word").count()

In [ ]:
# `complete`:All the rows in the streaming DataFrame/Dataset will be written to the sink
wordCount.writeStream.outputMode

## 밑에 실행하기 전에 nc -lvp 9999 실행할 것!! 

In [ ]:
# `complete`:All the rows in the streaming DataFrame/Dataset will be written to the sink
stream = wordCount.writeStream \
        .outputMode("complete")\
        .format("console")\
        .start()
    
stream.awaitTermination()

![image](https://user-images.githubusercontent.com/33097467/43726165-5aa5b7ea-99d9-11e8-9958-b8507353603d.png)

![image](https://user-images.githubusercontent.com/33097467/43726248-97b0b284-99d9-11e8-9a12-ed4c10e052ba.png)